<hr>

## Which neighbourhoods in NYC are most similar to each other?

#### Where are all the neighbourhoods in NYC?

In [1]:
# Original data source: https://geo.nyu.edu/catalog/nyu_2451_34572

import pandas as pd

nyc = pd.read_csv("nyc_neighborhoods.csv")

print(nyc.shape) #shows number of (rows, columns)

nyc.head(20) #returns first 20 rows

(306, 4)


,borough,neighborhood,latitude,longitude
0,Bronx,Allerton,40.865788,-73.859319
1,Staten Island,Annadale,40.538114,-74.178549
2,Staten Island,Arden Heights,40.549286,-74.185887
3,Staten Island,Arlington,40.635325,-74.165104
4,Staten Island,Arrochar,40.596313,-74.067124
5,Queens,Arverne,40.589144,-73.791992
6,Queens,Astoria,40.768509,-73.915654
7,Queens,Astoria Heights,40.770317,-73.894680
8,Queens,Auburndale,40.761730,-73.791762
9,Brooklyn,Bath Beach,40.599519,-73.998752


In [2]:
# focus on neighborhoods in Manhattan only
nyc = nyc[nyc.borough == "Manhattan"]

print(nyc.shape)
nyc.head(10)

(40, 4)


,borough,neighborhood,latitude,longitude
10,Manhattan,Battery Park City,40.711932,-74.016869
44,Manhattan,Carnegie Hill,40.782683,-73.953256
48,Manhattan,Central Harlem,40.815976,-73.943211
51,Manhattan,Chelsea (Manhattan),40.744035,-74.003116
52,Manhattan,Chinatown,40.715618,-73.994279
55,Manhattan,Civic Center,40.715229,-74.005415
59,Manhattan,Clinton,40.759101,-73.996119
80,Manhattan,East Harlem,40.792249,-73.944182
83,Manhattan,East Village,40.727847,-73.982226
97,Manhattan,Financial District,40.707107,-74.010665


## Plot the Manhattan neighborhoods on a map

#### Install the [folium](https://github.com/python-visualization/folium) package for plotting maps in Python

In [3]:
# Run this cell to install the folium package
# !pip install folium

#### Plot a map of Manhattan, New York
- latitude, longitude: 40.765937,-73.977304

In [4]:
import folium
import matplotlib.cm as cm

map_nyc = folium.Map(location=[40.765937,-73.977304], zoom_start=12)

map_nyc

#### Add markers for each neighborhood to the map:

In [5]:
map_nyc = folium.Map(location=[40.765937,-73.977304], zoom_start=12)

for HOOD, LAT, LON in zip(nyc.neighborhood, nyc.latitude, nyc.longitude):
    folium.CircleMarker([LAT, LON], 
                        popup = str(HOOD), #text
                        radius = 7).add_to(map_nyc)

map_nyc

#### Change the map style

"Stamen Toner"

In [6]:
map_nyc = folium.Map(location=[40.765937,-73.977304], tiles='Stamen Toner', zoom_start=12)#  tiles='Stamen Toner', zoom_start=12)


for HOOD, LAT, LON in zip(nyc.neighborhood, nyc.latitude, nyc.longitude):
    folium.CircleMarker([LAT, LON], 
                        popup = str(HOOD), #text
                        radius = 7,
                        color = "blue").add_to(map_nyc)

map_nyc

"Stamen Watercolor"

In [7]:
map_nyc = folium.Map(location=[40.765937,-73.977304], tiles='Stamen Watercolor', zoom_start=12)#  tiles='Stamen Toner', zoom_start=12)


for HOOD, LAT, LON in zip(nyc.neighborhood, nyc.latitude, nyc.longitude):
    folium.CircleMarker([LAT, LON], 
                        popup = str(HOOD), #text
                        radius = 7,
                        color = "blue").add_to(map_nyc)

map_nyc

<hr>

## Using the Foursquare API: Finding the profile of nearby venues

Please replace with your own ID/SECRET later by creating an account at: https://developer.foursquare.com

#### Credentials

In [8]:
CLIENT_ID = "F340AT21E3CPCGJ2W2C3PFLCHARCBSQ3MVFKHOMZFXVIC0NG"
CLIENT_SECRET = "4O5UVLS4HQKRDLF0TPDI0A0HD1MWAGVSSAPYDNOAHHTPYZRK"

#### Import libraries

In [9]:
import requests
from pandas.io.json import json_normalize
import pandas as pd
import json
pd.options.display.max_columns = 999

print("Libraries imported.")

Libraries imported.


### $function$ getNearestVenues: 
- Using the Foursquare API, fetch the nearby venues for each location in a list.
> `getNearestVenues(namelist, latlist, lonlist, radius=100, limit=100)`  
> - **namelist**: list of names of locations
> - **latlist**: list of latitudes
> - **lonlist**: list of longitudes
> - **radius**: (optional) in meters from lat/lon. Default = 100
> - **limit**: (optional) limit number of venues within radius to search. Default = 100  
> - _**Returns a dataframe of all the nearest venues for each input location**_

#### _Sample output:_
<img src=https://ibm.box.com/shared/static/hexljahyeqhkuv0slvc80xvqgyosmww4.png width=640 align=left>

In [10]:
import time

def getNearestVenues(namelist, latlist, lonlist, radius=500, limit=100):
    
    mylist=[]
    counter = 1
    totalcount = len(namelist)
    
    for NAME, LAT, LON in zip(namelist, latlist, lonlist):
        print(NAME)
        
        if (counter % 10 == 0) & (counter != 40):
            # Prevent exceeding rate limit of API requests
            print ("---Wait 3 sec--- [" + str(totalcount-counter) + " remaining]")
            time.sleep(3)
            
        # Set the API request url
        url = "https://api.foursquare.com/v2/venues/explore?&ll={}%2C%20{}\
                &intent=browse&radius={}\
                &limit={}\
                &client_id={}\
                &client_secret={}\
                &v=20161016"\
                .format(LAT, LON, radius, limit, CLIENT_ID, CLIENT_SECRET)
            
        # GET request
        results = requests.get(url).json()["response"]['groups']
        
        # Return specific items of interest from GET request result (for each nearby venue: name, lat, lon, category)
        mylist.append([(NAME, LAT, LON, 
                        v['venue']['name'], 
                        v['venue']['location']['lat'], 
                        v['venue']['location']['lng'],  
                        v['venue']['categories'][0]['name']) for v in results[0]['items']])
        counter += 1

    df = pd.DataFrame([item for sublist in mylist for item in sublist])
    df.columns = ['poi', 'poi_lat', 'poi_lon', 'venue', 'venue_lat', 'venue_lon', 'venue_cat']
    
    return(df)

Run the **getNearestVenues** function with `radius = 1000` meters:

In [11]:
# Returns at most 100 nearby venues for each of the borough's lon/lat within a radius of 1000 meters

nyc_venues = getNearestVenues(namelist = nyc.neighborhood,
                              latlist = nyc.latitude,
                              lonlist = nyc.longitude,
                              radius = 1000)


Battery Park City
Carnegie Hill
Central Harlem
Chelsea (Manhattan)
Chinatown
Civic Center
Clinton
East Harlem
East Village
Financial District
---Wait 3 sec--- [30 remaining]
Flatiron
Gramercy
Greenwich Village
Hamilton Heights
Hudson Yards
Inwood
Lenox Hill
Lincoln Square
Little Italy
Lower East Side
---Wait 3 sec--- [20 remaining]
Manhattan Valley
Manhattanville
Marble Hill
Midtown
Midtown South
Morningside Heights
Murray Hill (Manhattan)
Noho
Roosevelt Island
Soho
---Wait 3 sec--- [10 remaining]
Stuyvesant Town
Sutton Place
Tribeca
Tudor City
Turtle Bay
Upper East Side
Upper West Side
Washington Heights
West Village
Yorkville


In [12]:
print(nyc_venues.shape)
nyc_venues.head(20)

(3983, 7)


,poi,poi_lat,poi_lon,venue,venue_lat,venue_lon,venue_cat
0,Battery Park City,40.711932,-74.016869,Battery Park City Esplanade,40.711622,-74.017907,Park
1,Battery Park City,40.711932,-74.016869,Hudson Eats,40.712819,-74.015911,Food Court
2,Battery Park City,40.711932,-74.016869,"Waterfront Plaza, Brookfield Place",40.713159,-74.016062,Plaza
3,Battery Park City,40.711932,-74.016869,Brookfield Place,40.713204,-74.015619,Shopping Mall
4,Battery Park City,40.711932,-74.016869,Equinox Brookfield Place,40.712704,-74.014995,Gym / Fitness Center
5,Battery Park City,40.711932,-74.016869,One World Observatory,40.713282,-74.013972,Scenic Lookout
6,Battery Park City,40.711932,-74.016869,Mighty Quinn's BBQ,40.712740,-74.016018,BBQ Joint
7,Battery Park City,40.711932,-74.016869,Winter Garden Atrium,40.713242,-74.015854,Performing Arts Venue
8,Battery Park City,40.711932,-74.016869,Le District,40.713109,-74.015923,Market
9,Battery Park City,40.711932,-74.016869,National September 11 Memorial & Museum,40.711599,-74.012922,Memorial Site


### How many nearby venues were identified by Foursquare for each location ("point of interest")?

In [13]:
nyc_venues.groupby("poi").count()

,poi_lat,poi_lon,venue,venue_lat,venue_lon,venue_cat
poi,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,100,100,100,100,100,100
Chelsea (Manhattan),100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,100,100,100,100,100,100
East Village,100,100,100,100,100,100


## K-means clustering of neighborhoods based on their nearby venues

Given the dataframe, how can we do clustering to see which locations are most similar to each other?

In [14]:
nyc_venues.head()

,poi,poi_lat,poi_lon,venue,venue_lat,venue_lon,venue_cat
0,Battery Park City,40.711932,-74.016869,Battery Park City Esplanade,40.711622,-74.017907,Park
1,Battery Park City,40.711932,-74.016869,Hudson Eats,40.712819,-74.015911,Food Court
2,Battery Park City,40.711932,-74.016869,"Waterfront Plaza, Brookfield Place",40.713159,-74.016062,Plaza
3,Battery Park City,40.711932,-74.016869,Brookfield Place,40.713204,-74.015619,Shopping Mall
4,Battery Park City,40.711932,-74.016869,Equinox Brookfield Place,40.712704,-74.014995,Gym / Fitness Center


Answer: we cannot do clustering with this dataframe _as is_. Cannot cluster on the venue_cat column because it has categorical features. We need to reshape it.

**Goal:**
- one-hot encode the category column -- so that each venue category is its own category

In [15]:
# One hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['venue_cat']], prefix = "", prefix_sep="")

#add poi column back to dataframe, at the end
nyc_onehot["poi"] = nyc_venues.poi 

# Move poi column to the first column
col_names = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[col_names]

print(nyc_onehot.shape) # many new columns created!
nyc_onehot.head()

(3983, 298)


,poi,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restauran

### What's the "profile" for each neighborhood? Let's get the mean of each venue type by neighborhood

In [16]:
nyc_grouped = nyc_onehot.groupby('poi').mean().reset_index()
nyc_grouped.head(10)

,poi,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restauran

## Neighborhoods in Manhattan

_<font align=left>Image source: https://nickwoodhams.com/new-york-neighborhoods-by-proximity-to-manhattan/</font>  _

<img src=https://nickwoodhams.com/content/images/2015/07/Manhattan-neighborhoods-map.jpg width=350 align=left>  



### What are the most popular venues for each neighborhood?

In [17]:
for hood in nyc_grouped.poi:
    print("----"+hood+"----")
    temp = nyc_grouped.loc[nyc_grouped.poi == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.ix[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(3))
    print("\n")

----Battery Park City----
                  venue  freq
0                  Park  0.06
1           Coffee Shop  0.06
2  Gym / Fitness Center  0.04


----Carnegie Hill----
                  venue  freq
0                  Café  0.07
1                Bakery  0.05
2  Gym / Fitness Center  0.05


----Central Harlem----
                             venue  freq
0                             Café  0.06
1  Southern / Soul Food Restaurant  0.05
2             Gym / Fitness Center  0.04




C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


----Chelsea (Manhattan)----
                venue  freq
0         Art Gallery  0.12
1               Hotel  0.05
2  Italian Restaurant  0.04


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.07
1      Ice Cream Shop  0.06
2        Cocktail Bar  0.05


----Civic Center----
               venue  freq
0             Bakery  0.05
1              Hotel  0.05
2  French Restaurant  0.05


----Clinton----
                venue  freq
0             Theater  0.07
1  Italian Restaurant  0.06
2               Hotel  0.05


----East Harlem----
                venue  freq
0  Mexican Restaurant  0.09
1         Pizza Place  0.07
2              Bakery  0.06


----East Village----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.07
2  Ice Cream Shop  0.06


----Financial District----
         venue  freq
0  Coffee Shop  0.09
1        Hotel  0.06
2         Park  0.05


----Flatiron----
                     venue  freq
0     Gym / Fitness Center  0.06
1             Cy

### View top venues per neighborhood as a table

In [18]:
top_n = 3

df = pd.DataFrame({n: nyc_grouped.drop("poi", axis=1).T[col].nlargest(top_n).index.tolist() 
                  for n, col in enumerate(nyc_grouped.T)}).T
df["poi"] = nyc_grouped.poi
df[df.columns[-1:].union(df.columns[:-1])].head()

,poi,0,1,2
0,Battery Park City,Coffee Shop,Park,American Restaurant
1,Carnegie Hill,Café,Bakery,Gym / Fitness Center
2,Central Harlem,Café,Southern / Soul Food Restaurant,African Restaurant
3,Chelsea (Manhattan),Art Gallery,Hotel,American Restaurant
4,Chinatown,Chinese Restaurant,Ice Cream Shop,Cocktail Bar


<hr>

## K-means clustering

#### Review our dataframe:
Because each variable has a numeric value, we can now cluster the neighborhoods together.

In [19]:
nyc_grouped.head()

,poi,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restauran

### **Goal**:
Use k-means clustering to group similar POI's together into clusters.

In [20]:
from sklearn.cluster import KMeans

# Set number of clusters
kclusters = 5

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped.drop('poi', 1))

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 0, 2, 4, 4, 4, 0, 3, 1])

#### Combine the cluster labels with the actual data

In [21]:
nyc_merged = nyc

# Add clustering labels
nyc_merged['cluster_labels'] = kmeans.labels_

# Merge nyc_grouped with nyc to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.merge(nyc_grouped, left_on="neighborhood", right_on="poi")

nyc_merged.head() #Check the last columns!

# TWK -> look at cluser_labels column

,borough,neighborhood,latitude,longitude,cluster_labels,poi,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cable Car,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Th

## Plot the neighborhoods on a map, <font color="blue">coloured</font> by its cluster

In [22]:
import folium
import matplotlib.cm as cm
import numpy as np
import matplotlib.colors as colors

# Create map
map_clusters = folium.Map(location=[40.765937,-73.977304], zoom_start=13)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
for POI, LAT, LON, CLU in zip(nyc_merged.poi, nyc_merged.latitude, nyc_merged.longitude, nyc_merged.cluster_labels):
    folium.CircleMarker([LAT, LON], 
                        popup = str(POI)+" Cluster "+str(CLU),
                        radius = 7, 
                        color = rainbow[CLU-1], 
                        fill_color = rainbow[CLU-1],
                        fill_opacity = 0.7).add_to(map_clusters)

map_clusters

<hr>